In [2]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=4)

# Exercise 09:  model selection

* Select a dataset and run nested cross validation to estimate the performance of a nonlinear SVM classifier, with grid search performed over the value of gamma and C.  
* Compare the accuracy you get with nested cross-validation with the performance you obtain with the default parameters.  What parameters are chosen when you train a GridSearchCV object?  
* In my experience, you can usually get performance which is similar by performing search only on the value of gamma.  Does that pan out for the dataset you have chosen?

In [9]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn import svm

data = load_breast_cancer()
X = data.data
y = data.target


In [11]:
from sklearn.model_selection import train_test_split
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
model = svm.SVC(kernel="rbf")
model.fit(X_train,y_train)
y_pred = model.predict(X_test)


In [22]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

outerfolds = StratifiedKFold (n_splits = 5)
outerfolds.get_n_splits(X)
# outer_cv
fold_outer = 0;
for train, test in outerfolds.split(X,y):
  X_train, X_test, y_train, y_test = X[train],X[test],y[train],y[test]
  model = svm.SVC(kernel="rbf")
  model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  fold_outer +=1
  print(f"Accuracy in outer fold {fold_outer}:", accuracy_score(y_pred, y_test))


Accuracy in outer fold 1: 0.8508771929824561
Accuracy in outer fold 2: 0.8947368421052632
Accuracy in outer fold 3: 0.9298245614035088
Accuracy in outer fold 4: 0.9473684210526315
Accuracy in outer fold 5: 0.9380530973451328


In [27]:
#inner_cv
innerfolds = StratifiedKFold (n_splits = 5)
innerfolds.get_n_splits(X)
fold_inner = 0;
for train, valid in innerfolds.split(X_train,y_train):
  X_train, X_valid, y_train, y_valid = X[train],X[valid],y[train],y[valid]
  model = svm.SVC(kernel="rbf")
  model.fit(X_train,y_train)
  y_pred = model.predict(X_valid)
  fold_inner +=1
  print(f"Accuracy in inner fold {fold_inner}:", accuracy_score(y_pred, y_valid))

Accuracy in inner fold 1: 0.8936170212765957
Accuracy in inner fold 2: 0.8085106382978723
Accuracy in inner fold 3: 0.9574468085106383
Accuracy in inner fold 4: 0.8936170212765957
Accuracy in inner fold 5: 0.8260869565217391


In [23]:
from sklearn.model_selection import GridSearchCV

param_grid = [
  {'C': [1, 10, 100, 1000], 
   'gamma': [0.001, 0.01, 0.1], 
   'kernel': ['rbf']},
 ]

model = GridSearchCV(svm.SVC(), param_grid)

model.fit(X, y);
model.best_estimator_

SVC(C=1, gamma=0.001)

In [46]:
outer_scores = np.zeros(5)
nested_scores = np.zeros(5)

for i in range (5):
  outerfolds = StratifiedKFold (n_splits = 5)
  innerfolds = StratifiedKFold (n_splits = 5)
  m1 = svm.SVC(kernel="rbf")
  model = GridSearchCV(estimator= m1, param_grid = param_grid, cv = innerfolds)
  model.fit(X,y)
  # outer_scores[i] = model.best_estimator_
  outer_scores[i] = model.best_score_

  nested_scores = cross_val_score(model, X=X,y=y, cv = outerfolds)
  nested_scores[i] = nested_scores.mean()
  print(f"Mean {i+1}:", nested_scores.mean())


Mean 1: 0.9211799410029498
Mean 2: 0.922934326967862
Mean 3: 0.922934326967862
Mean 4: 0.9176711690731253
Mean 5: 0.9283837913367489


In [52]:
from numpy.core.numeric import outer
score_difference = outer_scores - nested_scores

print("Score difference from outer and inner", score_difference, "Absolute average of differnce: ",abs(score_difference.mean()))

Score difference from outer and inner [-0.0072  0.0016  0.0016 -0.0247  0.    ] Absolute average of differnce:  0.005763080267039267


As the reuslt, since i mean the score of innerfolds and compare them with the scores of outerfolds.So, there is no bearing on the signficance of the difference between groups. So the dataset what I have chosen, it pan out.